In [1]:
import os
from pathlib import Path
# from src.dataloader import DataLoader

ROOT = Path(os.getcwd()).parents[0]

ROOT

PosixPath('/Users/dantrainer/projects/live-bib-tracking')

In [7]:
def move_json_files(source_dir, target_dir="./labels"):
    """
    Scans a directory and moves all .json files to a target directory.

    Args:
        source_dir (str): The directory to scan for .json files.
        target_dir (str): The directory where .json files will be moved. Defaults to './labels'.
    """
    os.makedirs(target_dir, exist_ok=True)  # Create the target directory if it doesn't exist

    for file_name in os.listdir(source_dir):
        if file_name.endswith(".json"):
            source_path = os.path.join(source_dir, file_name)
            target_path = os.path.join(target_dir, file_name)
            os.rename(source_path, target_path)  # Move the file

data_path = os.path.join(ROOT, "data", "processed","frames")

move_json_files(data_path, target_dir=os.path.join(ROOT, "data", "processed", "labels"))

In [9]:
import os
import json
from glob import glob

# Paths
labelme_dir = os.path.join(ROOT, "data", "processed", "annotations")
output_dir = os.path.join(ROOT, "data", "processed", "labels")
os.makedirs(output_dir, exist_ok=True)

# Loop over JSON files
for json_file in glob(os.path.join(labelme_dir, "*.json")):
    with open(json_file, "r") as f:
        data = json.load(f)

    img_w = data["imageWidth"]
    img_h = data["imageHeight"]

    yolo_lines = []
    for shape in data["shapes"]:
        label = int(shape["label"])  # assuming labels are numeric in your JSON

        # LabelMe rectangle has two points: top-left and bottom-right
        (x1, y1), (x2, y2) = shape["points"]

        # Calculate YOLO format
        x_center = ((x1 + x2) / 2) / img_w
        y_center = ((y1 + y2) / 2) / img_h
        width = abs(x2 - x1) / img_w
        height = abs(y2 - y1) / img_h

        yolo_lines.append(f"{label} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # Save .txt with same base name as the image
    base_name = os.path.splitext(os.path.basename(data["imagePath"]))[0]
    txt_path = os.path.join(output_dir, f"{base_name}.txt")

    with open(txt_path, "w") as out_f:
        out_f.write("\n".join(yolo_lines))

    print(f"Converted {json_file} → {txt_path}")

print(f"\n✅ Done. YOLO labels saved in '{output_dir}'")


Converted /Users/dantrainer/projects/live-bib-tracking/data/processed/annotations/frame_00876.json → /Users/dantrainer/projects/live-bib-tracking/data/processed/labels/frame_00876.txt
Converted /Users/dantrainer/projects/live-bib-tracking/data/processed/annotations/frame_00437.json → /Users/dantrainer/projects/live-bib-tracking/data/processed/labels/frame_00437.txt
Converted /Users/dantrainer/projects/live-bib-tracking/data/processed/annotations/frame_00421.json → /Users/dantrainer/projects/live-bib-tracking/data/processed/labels/frame_00421.txt
Converted /Users/dantrainer/projects/live-bib-tracking/data/processed/annotations/frame_00564.json → /Users/dantrainer/projects/live-bib-tracking/data/processed/labels/frame_00564.txt
Converted /Users/dantrainer/projects/live-bib-tracking/data/processed/annotations/frame_00163.json → /Users/dantrainer/projects/live-bib-tracking/data/processed/labels/frame_00163.txt
Converted /Users/dantrainer/projects/live-bib-tracking/data/processed/annotation

In [3]:
import os
import json
import torch
from torch.utils.data import Dataset

from PIL import Image


class ObjectDetectionDataset(Dataset):
    """Dataset for object detection tasks with images and annotations."""
    def __init__(self, root_dir, transform=None, test=True):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        self.annotation_files = []
        self.test = test

        # Populate image and annotation file lists
        for filename in os.listdir(root_dir):
            if not filename.endswith(".json") and not self.test:
                continue

            root_filename = filename.split(".")[0]

            image_file = os.path.join(root_dir, root_filename + ".jpg")
            annotation_file = os.path.join(root_dir, filename)

            if os.path.exists(image_file):
                self.image_files.append(image_file)
                self.annotation_files.append(annotation_file)

    def _read_and_validate_annotation_file(json_file):
        with open(json_file, "r") as file:
            annotation_file = json.load(file)

        shapes = annotation_file["shapes"]

        assert len(shapes) == 2, "more than 2 labels provided"
        assert "0" in [x["label"] for x in shapes], "no grid label provided"
        assert "1" in [x["label"] for x in shapes], "no cross hair label provided"

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img = Image.open(img_path).convert("RGB")
        initial_size = img.size

        annotation_path = self.annotation_files[idx]
        boxes = []
        labels = []
        filename = img_path.split("/")[-1]
        if not self.test:
            with open(annotation_path, "r") as file:
                annotations = json.load(file)

            for line in annotations["shapes"]:
                boxes.append(line["points"])
                labels.append(int(line["label"]))

            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)

            cross_hair_box = boxes[1]

            x1 = cross_hair_box[0, 0]
            y1 = cross_hair_box[0, 1]
            x2 = cross_hair_box[1, 0]
            y2 = cross_hair_box[1, 1]

            x_box = torch.tensor(((x1, 0), (x2, img.size[1])))
            y_box = torch.tensor(((0, y1), (img.size[0], y2)))

            boxes = torch.stack((boxes[0], x_box, y_box))
            labels = torch.concat((labels, torch.tensor((2,))))

        if self.transform:
            img = self.transform(img)
            resized_shape = img.size()
            scale_factor = initial_size[0] / resized_shape[2]

            if len(boxes) != 0:
                boxes /= scale_factor

        return img, boxes, labels, filename

In [4]:
data_path = os.path.join(ROOT, "data", "processed","frames")
train_dataset = ObjectDetectionDataset(data_path, test=False)

In [ ]:
img, boxes, labels, filename = train_dataset.__getitem__(-1)

In [6]:
img, boxes, labels, filename

(<PIL.Image.Image image mode=RGB size=3840x2160>,
 tensor([[[2801.9148,  791.7021],
          [3418.9363, 2147.0212]],
 
         [[3095.5320,    0.0000],
          [3263.6169, 2160.0000]],
 
         [[   0.0000, 1302.3405],
          [3840.0000, 1451.2766]]]),
 tensor([0, 1, 0, 0, 2, 2, 1, 2]),
 'frame_00361.jpg')